<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [2]:
# Ensure the right version of Tensorflow is installed.
!pip freeze | grep tensorflow==2.1

In [4]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-00-0fd1606d0057'
PROJECT = 'qwiklabs-gcp-00-0fd1606d0057'
REGION = 'us-central1'

In [5]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [6]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [7]:
# Create SQL query using natality data after the year 2000
from google.cloud import bigquery
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [8]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bigquery.Client().query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,-7170969733900686954,331274
1,5896567601480310696,342825
2,-5742197815970064689,359684
3,-774501970389208065,349134
4,454960867574323744,346515


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [9]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) = 3 AND RAND() < 0.0005"
traindf = bigquery.Client().query(trainQuery).to_dataframe()
evaldf = bigquery.Client().query(evalQuery).to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13234 examples in the train dataset and 3364 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [10]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.583902,True,30,1,39.0,3095933535584005890
1,7.436192,False,20,1,40.0,3095933535584005890
2,6.437498,True,31,1,37.0,3095933535584005890
3,7.438397,False,20,1,41.0,3095933535584005890
4,4.812691,True,22,1,38.0,3095933535584005890


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [11]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13229.000000,13234.000000,13234.000000,13144.000000,1.323400e+04
mean,7.223298,27.351821,1.034457,38.604002,2.635746e+17
std,1.325436,6.166019,0.197905,2.570823,5.238524e+18
min,0.562179,12.000000,1.000000,17.000000,-9.183606e+18
25%,6.563162,22.000000,1.000000,38.000000,-4.329667e+18
50%,7.312733,27.000000,1.000000,39.000000,-3.280124e+17
75%,8.046873,32.000000,1.000000,40.000000,4.331750e+18
max,12.936726,50.000000,5.000000,47.000000,8.599690e+18


In [12]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [13]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.583902,True,30,Single(1),39.0,3095933535584005890
1,7.436192,False,20,Single(1),40.0,3095933535584005890
2,6.437498,True,31,Single(1),37.0,3095933535584005890
3,7.438397,False,20,Single(1),41.0,3095933535584005890
4,4.812691,True,22,Single(1),38.0,3095933535584005890


In [14]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13229,7.874912,Unknown,20,Single(1),39.0,-774501970389208065
13230,7.705156,Unknown,22,Single(1),40.0,-774501970389208065
13231,7.063611,Unknown,39,Single(1),38.0,-774501970389208065
13232,8.187968,Unknown,27,Single(1),38.0,-774501970389208065
13233,8.811877,Unknown,36,Single(1),39.0,-774501970389208065


In [15]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26278.000000,26278.000000,26278.000000,2.627800e+04
mean,7.222999,27.347743,38.603623,2.604770e+17
std,1.326499,6.163569,2.570521,5.235411e+18
min,0.562179,12.000000,17.000000,-9.183606e+18
25%,6.563162,22.000000,38.000000,-4.329667e+18
50%,7.312733,27.000000,39.000000,-3.280124e+17
75%,8.050731,32.000000,40.000000,4.331750e+18
max,12.936726,50.000000,47.000000,8.599690e+18


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [16]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [17]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

   6682 eval.csv
  26278 train.csv
  32960 total
==> eval.csv <==
6.87621795178,True,22,Single(1),37.0,8904940584331855459
6.9996768185,False,16,Single(1),39.0,6392072535155213407
7.8153871879,False,21,Single(1),36.0,-1639186255933990135
8.68841774542,True,36,Single(1),39.0,-6141045177192779423
7.25100379718,False,20,Single(1),39.0,-7146494315947640619
8.9617909503,False,33,Single(1),40.0,74931465496927487
7.054792384,False,31,Single(1),37.0,270792696282171059
6.93794738514,True,27,Single(1),37.0,2246942437170405963
6.75055446244,True,19,Single(1),38.0,270792696282171059
6.4992274837599995,True,22,Single(1),40.0,-7517141034410775575

==> train.csv <==
7.5839018128,True,30,Single(1),39.0,3095933535584005890
7.43619209726,False,20,Single(1),40.0,3095933535584005890
6.4374980503999994,True,31,Single(1),37.0,3095933535584005890
7.43839671988,False,20,Single(1),41.0,3095933535584005890
4.81269117946,True,22,Single(1),38.0,3095933535584005890
7.3744626639,True,25,Single(1),41.0,3095933535584

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License